### Compute OFI Metrics:
- Derive multi-level OFI metrics (up to 5 levels) for each stock in the dataset.
- Integrate these multi-level OFIs into a single metric using Principal Component Analysis (PCA) or another dimensionality reduction method.

In [2]:
import pandas as pd
import numpy as np
import glob
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import os
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [9]:
# file_paths = sorted(glob.glob('../data/*.csv.zst'))  # Sort files by filename
# data_list = [pd.read_csv(file_paths[i]) for i in range(5)]
# data = pd.concat(data_list, ignore_index=True)
data = pd.read_csv("../data/raw_data/dbeq-basic-20241216.mbp-10.csv.zst")

In [10]:
def process_minute_level(data):
    """
    Process data at the minute level by keeping the last state within each minute.

    Args:
        data (pd.DataFrame): DataFrame containing order book data.

    Returns:
        pd.DataFrame: Minute-level processed data.
    """
    data.loc[:, 'timestamp'] = pd.to_datetime(data.loc[:,'ts_recv']).dt.floor('T')
    # Use groupby and last directly without sorting
    minute_level_data = data.groupby(['symbol', 'timestamp']).last().reset_index()
    minute_level_data.fillna(0, inplace=True)
    return minute_level_data


In [11]:
required_columns = [
    'ts_recv',    'action',
    'side',
    'price',
    'size',
    'sequence',
    'bid_px_00',
    'ask_px_00',
    'bid_sz_00',
    'ask_sz_00',
    'bid_ct_00',
    'ask_ct_00',
    'bid_px_01',
    'ask_px_01',
    'bid_sz_01',
    'ask_sz_01',
    'bid_ct_01',
    'ask_ct_01',
    'bid_px_02',
    'ask_px_02',
    'bid_sz_02',
    'ask_sz_02',
    'bid_ct_02',
    'ask_ct_02',
    'bid_px_03',
    'ask_px_03',
    'bid_sz_03',
    'ask_sz_03',
    'bid_ct_03',
    'ask_ct_03',
    'bid_px_04',
    'ask_px_04',
    'bid_sz_04',
    'ask_sz_04',
    'bid_ct_04',
    'ask_ct_04',
    'symbol',
]


In [12]:
# df = data[required_columns]
# df['timestamp'] = pd.to_datetime(data.loc[:, 'ts_recv']).dt.floor('T')
minute_level_data = process_minute_level(data[required_columns])
minute_level_data.head(10)

/var/folders/yj/jjmvh7dd6sx95p4gzltx4j2r0000gn/T/ipykernel_16293/1560544773.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'timestamp'] = pd.to_datetime(data.loc[:,'ts_recv']).dt.floor('T')


,symbol,timestamp,ts_recv,action,side,price,size,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,bid_px_02,ask_px_02,bid_sz_02,ask_sz_02,bid_ct_02,ask_ct_02,bid_px_03,ask_px_03,bid_sz_03,ask_sz_03,bid_ct_03,ask_ct_03,bid_px_04,ask_px_04,bid_sz_04,ask_sz_04,bid_ct_04,ask_ct_04
0,AAPL,2024-12-16 07:03:00+00:00,2024-12-16T07:03:00.029417437Z,R,N,0.0,0,7247,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
1,AAPL,2024-12-16 07:04:00+00:00,2024-12-16T07:04:00.028766595Z,R,N,0.0,0,7114,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
2,AAPL,2024-12-16 12:00:00+00:00,2024-12-16T12:00:51.236171281Z,A,B,247.6,2000,15333,247.6,249.4,2000,2000,1,1,0.0,249.6,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
3,AAPL,2024-12-16 12:01:00+00:00,2024-12-16T12:01:38.629222193Z,A,A,249.6,2000,16126,247.6,249.6,2000,2000,1,1,247.2,250.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
4,AAPL,2024-12-16 12:02:00+00:00,2024-12-16T12:02:57.178549321Z,A,A,249.4,2000,17143,247.6,249.4,2000,2000,1,1,247.4,249.8,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
5,AAPL,2024-12-16 12:03:00+00:00,2024-12-16T12:03:21.144528379Z,A,B,247.4,2000,17349,247.4,249.4,2000,2000,1,1,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
6,AAPL,2024-12-16 12:04:00+00:00,2024-12-16T12:04:46.939932329Z,C,B,247.4,2000,18103,247.6,249.4,2000,2000,1,1,247.4,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
7,AAPL,2024-12-16 12:05:00+00:00,2024-12-16T12:05:30.815491776Z,C,A,249.8,2000,18535,247.6,249.6,2000,2000,1,1,0.0,249.8,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
8,AAPL,2024-12-16 12:06:00+00:00,2024-12-16T12:06:16.522543203Z,C,B,247.2,2000,19155,247.6,249.6,2000,2000,1,1,247.2,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0
9,AAPL,2024-12-16 12:07:00+00:00,2024-12-16T12:07:41.238933899Z,A,A,249.8,2000,19897,247.4,249.8,2000,2000,1,1,247.0,249.6,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0


In [14]:
def compute_ofi(data, levels=5):
    """
    Compute multi-level OFI metrics.
    
    Args:
        data (pd.DataFrame): Input data containing LOB states.
        levels (int): Number of levels to compute OFI for.

    Returns:
        pd.DataFrame: DataFrame containing OFI metrics for each stock.
    """
    ofi_results = []

    for symbol, group in data.groupby('symbol'):
        # Calculate mid-price
        group['mid_price'] = (group['bid_px_00'] + group['ask_px_00']) / 2

        # Filter out rows where mid_price is zero
        group = group[group['mid_price'] > 0].reset_index(drop=True)

        # Initialize dictionary for storing OFI and mid-price
        symbol_ofi = {
            'symbol': group['symbol'],
            'timestamp': group['timestamp'],
            'mid_price': group['mid_price']
        }

        for level in range(0, levels):
            bid_price_col = f'bid_px_0{level}'
            bid_size_col = f'bid_sz_0{level}'
            ask_price_col = f'ask_px_0{level}'
            ask_size_col = f'ask_sz_0{level}'

            # Compute bid and ask OFIs
            bid_ofi = np.where(
                group[bid_price_col].diff() > 0, group[bid_size_col],
                np.where(
                    group[bid_price_col].diff() == 0,
                    group[bid_size_col].diff(),
                    -group[bid_size_col]
                )
            )

            ask_ofi = np.where(
                group[ask_price_col].diff() > 0, -group[ask_size_col],
                np.where(
                    group[ask_price_col].diff() == 0,
                    group[ask_size_col].diff(),
                    group[ask_size_col]
                )
            )

            level_ofi = bid_ofi - ask_ofi
            symbol_ofi[f'ofi_level_{level}'] = level_ofi

        # Convert to DataFrame and append to results
        ofi_results.append(pd.DataFrame(symbol_ofi))

    return pd.concat(ofi_results, ignore_index=True)

ofi_metrics = compute_ofi(minute_level_data, levels=5)
# Save the results
ofi_metrics.to_csv('../data/ofi_metrics/20241216_ofi_metrics.csv', index=False)

print("OFI metrics have been successfully computed and saved.")


OFI metrics have been successfully computed and saved.


In [40]:
def integrate_ofi_with_pca(daily_data, date, levels=5):
    """
    Integrate multi-level OFIs into a single metric using PCA for each stock in a given trading day.

    Args:
        daily_data (pd.DataFrame): DataFrame containing the daily OFI metrics.
        date (str): Trading day date as a string.
        levels (int): Number of levels used in OFI computation.

    Returns:
        pd.DataFrame: Updated DataFrame with integrated OFI and explained variance.
        list: A list of dictionaries containing explained variance data for each stock on the given day.
    """
    explained_variances = []

    for symbol, group in daily_data.groupby('symbol'):
        pca_columns = [f'ofi_level_{i}' for i in range(levels)]
        original_indices = group.index  # Track original indices for correct assignment later

        # Check if all columns are null or constant
        if group[pca_columns].isnull().all().any() or group[pca_columns].nunique().max() == 1:
            print(f"Skipping PCA for symbol {symbol} on {date}: insufficient data.")
            daily_data.loc[original_indices, 'integrated_ofi'] = np.nan
            explained_variances.append({
                'date': date,
                'symbol': symbol,
                'explained_variance': np.nan
            })
            continue

        # Perform PCA on multi-level OFI columns
        pca = PCA(n_components=1)
        group['integrated_ofi'] = pca.fit_transform(group[pca_columns])

        # Update the original daily_data DataFrame with the calculated 'integrated_ofi'
        daily_data.loc[original_indices, 'integrated_ofi'] = group['integrated_ofi']

        # Collect explained variance
        explained_variances.append({
            'date': date,
            'symbol': symbol,
            'explained_variance': pca.explained_variance_ratio_[0]
        })

    return daily_data, explained_variances


In [42]:
file_paths = glob.glob("../data/ofi_metrics/*_ofi_metrics.csv")
all_explained_variances = []

for file_path in file_paths:
    date = file_path.split('/')[-1].split('_')[0]  # Extract date from filename
    daily_data = pd.read_csv(file_path)
    daily_data, daily_explained_variances = integrate_ofi_with_pca(daily_data, date, levels=5)

    # Save updated daily data with integrated OFI
    daily_data.to_csv(file_path, index=False)

    # Collect explained variance data
    all_explained_variances.extend(daily_explained_variances)
    
explained_variance_df = pd.DataFrame(all_explained_variances)
explained_variance_matrix = explained_variance_df.pivot(index='date', columns='symbol', values='explained_variance')
explained_variance_matrix.to_csv('../data/ofi_metrics/explained_variance_matrix.csv')

### Analyze Cross-Impact:
- Examine the contemporaneous cross-impact of OFI on short-term price changes
across stocks.
- Evaluate the predictive power of lagged cross-asset OFI on future price changes
(e.g., 1-minute and 5-minute horizons).

In [144]:
def analyze_contemporaneous_cross_impact(daily_data):
    """
    Analyze contemporaneous cross-impact of OFI on short-term price changes across stocks.

    Args:
        daily_data (pd.DataFrame): DataFrame containing the daily OFI metrics.

    Returns:
        float: R^2 value of cross-impact analysis for the given day.
    """
    results = []

    # Prepare integrated OFI data for all stocks
    integrated_ofi = daily_data.pivot(index='timestamp', columns='symbol', values='integrated_ofi')
    mid_prices = daily_data.pivot(index='timestamp', columns='symbol', values='mid_price')
    mid_price_changes = np.log(mid_prices).diff()[1:]

    # Loop through each stock for individual models
    for stock in mid_price_changes.columns:
        Y = mid_price_changes[stock].dropna()
        X = integrated_ofi.loc[Y.index].ffill().bfill()  # Match X and Y by index

        if X.empty or Y.empty:
            print(f"Skipping stock {stock}: insufficient data for regression.")
            continue

        model = LinearRegression()
        model.fit(X, Y)

        r2 = r2_score(Y, model.predict(X))
        coefficients = model.coef_
        intercept = model.intercept_

        result = {'stock': stock, 'r2': r2, 'intercept': intercept}
        for idx, coeff in enumerate(coefficients):
            result[f'coef_{integrated_ofi.columns[idx]}'] = coeff

        results.append(result)

    return results

In [172]:
file_paths = glob.glob("../data/ofi_metrics/*_ofi_metrics.csv")

for file_path in file_paths:
    date = file_path.split('/')[-1].split('_')[0]  # Extract date from filename
    # print(date)
    daily_data = pd.read_csv(file_path)
    regression_results = analyze_contemporaneous_cross_impact(daily_data)
    regression_results_df = pd.DataFrame(regression_results)
    # Save updated daily data with integrated OFI
    regression_results_df.to_csv(f"../data/contemporaneous_regression_results/{date}_result.csv")

In [158]:
def analyze_predictive_power(daily_data, horizons=[1, 5]):
    """
    Evaluate the predictive power of lagged cross-asset OFI on future price changes.

    Args:
        daily_data (pd.DataFrame): DataFrame containing the daily OFI metrics.
        horizons (list): List of time horizons (in minutes) for predictive analysis.

    Returns:
        pd.DataFrame: DataFrame containing R^2 values, coefficients, and intercepts for each stock and horizon.
    """
    results = []

    # Prepare integrated OFI data for all stocks
    integrated_ofi = daily_data.pivot(index='timestamp', columns='symbol', values='integrated_ofi')
    mid_prices = daily_data.pivot(index='timestamp', columns='symbol', values='mid_price')

    for horizon in horizons:
        future_price_changes = np.log(mid_prices.shift(-horizon)) - np.log(mid_prices)

        for stock in future_price_changes.columns:
            Y = future_price_changes[stock].dropna()
            X = integrated_ofi.loc[Y.index].ffill().bfill()  # Match X and Y by index

            if X.empty or Y.empty:
                print(f"Skipping stock {stock} for horizon {horizon}: insufficient data for regression.")
                continue

            model = LinearRegression()
            model.fit(X, Y)

            r2 = r2_score(Y, model.predict(X))
            coefficients = model.coef_
            intercept = model.intercept_

            result = {'stock': stock, 'horizon': horizon, 'r2': r2, 'intercept': intercept}
            for idx, coeff in enumerate(coefficients):
                result[f'coef_{integrated_ofi.columns[idx]}'] = coeff

            results.append(result)

    return results


In [173]:
file_paths = glob.glob("../data/ofi_metrics/*_ofi_metrics.csv")

for file_path in file_paths:
    date = file_path.split('/')[-1].split('_')[0]  # Extract date from filename
    # print(date)
    daily_data = pd.read_csv(file_path)
    regression_results = analyze_predictive_power(daily_data)
    regression_results_df = pd.DataFrame(regression_results)
    # Save updated daily data with integrated OFI
    regression_results_df.to_csv(f"../data/predictive_regression_results/{date}_result.csv")

### Quantify Results:
- Use regression models to assess the explanatory power of contemporaneous
OFI and predictive power of lagged OFI.
- Compare self-impact (within the same stock) vs. cross-impact (between stocks)
in your models.

In [209]:
def evaluate_explanatory_power(cross_impact_results):
    """
    Evaluate explanatory power of contemporaneous OFI models.

    Args:
        cross_impact_results (pd.DataFrame): DataFrame containing R^2 and coefficients from contemporaneous models.

    Returns:
        pd.DataFrame: Aggregated metrics for explanatory power.
    """
    grouped = cross_impact_results.groupby('stock').mean()

    # Prepare a readable matrix output
    summary_matrix = grouped.filter(like='coef_').copy()
    summary_matrix['avg_r2'] = grouped['r2']

    return summary_matrix.reset_index()


def evaluate_predictive_power(predictive_power_results):
    """
    Evaluate predictive power of lagged OFI models.

    Args:
        predictive_power_results (pd.DataFrame): DataFrame containing R^2 and coefficients from predictive models.
        output_excel_path (str): Path to save the Excel file with results for each horizon.

    Returns:
        None
    """
    summary_matrix_dictionary = {}
    for horizon in predictive_power_results['horizon'].unique():
        subset = predictive_power_results[predictive_power_results['horizon'] == horizon]
        explanatory_summary = evaluate_explanatory_power(subset)
        summary_matrix_dictionary[horizon] = explanatory_summary

    return summary_matrix_dictionary


In [ ]:
file_paths = glob.glob("../data/contemporaneous_regression_results/*_result.csv")
cross_impact_results = []

for file_path in file_paths:
    date = file_path.split('/')[-1].split('_')[0]  # Extract date from filename
    daily_result = pd.read_csv(file_path)
    cross_impact_results.append(daily_result)
cross_impact_combined = pd.concat(cross_impact_results, ignore_index = True)
cross_impact_combined = cross_impact_combined.loc[:, ~cross_impact_combined.columns.str.contains('^Unnamed')]
explanatory_summary = evaluate_explanatory_power(cross_impact_combined)
explanatory_summary.to_csv('../data/contemporaneous_regression_results/explanatory_power_summary.csv', index=False)


In [215]:
file_paths = glob.glob("../data/predictive_regression_results/*_result.csv")
predictive_regression_results = []

for file_path in file_paths:
    date = file_path.split('/')[-1].split('_')[0]  # Extract date from filename
    daily_result = pd.read_csv(file_path)
    predictive_regression_results.append(daily_result)

predictive_power_combined = pd.concat(predictive_regression_results, ignore_index = True)
predictive_power_combined = predictive_power_combined.loc[:, ~predictive_power_combined.columns.str.contains('^Unnamed')]
predictive_summary_dict = evaluate_predictive_power(predictive_power_combined)

output_excel_path = "../data/predictive_regression_results/predictive_power_summary.xlsx"
os.makedirs(os.path.dirname(output_excel_path), exist_ok=True)

with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        for horizon in predictive_summary_dict:
            subset = predictive_summary_dict[horizon]
            subset.to_excel(writer, sheet_name=f'Horizon_{horizon}', index=False)


### Visualization
- Create visualizations (e.g., heatmaps, scatter plots) to illustrate cross-impact relationships and OFI trends.

In [12]:
def visualize_heatmap(data, title="heatmap", output_dir="../results/"):
    """
    Generate a heatmap image based on the given dataframe.

    Args:
        data (pd.DataFrame): Summary DataFrame.
        title (str): Title for the heatmap and filename.
        output_dir (str): Directory to save the heatmap.

    Returns:
        None
    """
    # Prepare output path
    output_path = os.path.join(output_dir, f"{title} Heatmap.png")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Prepare data for heatmap
    heatmap_data = data.set_index('stock').filter(like='coef_').fillna(0)
    
    # Format values for annotations
    annotation_data = heatmap_data.applymap(lambda x: f"{x:.2e}")
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        heatmap_data,
        annot=annotation_data,
        fmt="",
        cmap="coolwarm",
        cbar_kws={"format": "%.1e"},
        linewidths=0.5
    )
    plt.title(title)
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for better readability
    plt.yticks(rotation=0)  # Keep y-axis labels horizontal
    plt.tight_layout()  # Adjust layout to prevent clipping
    plt.savefig(output_path)
    plt.close()
    print(f"Heatmap saved to {output_path}.")


def visualize_scatter_plot(data, x_variable, y_variable, title = "scatter_plot", output_dir="../results/"):
    """
    Generate scatter plots for predictive power results with X as time horizon, Y as R^2, and legend as different stocks.

    Args:
        predictive_power_results (pd.DataFrame): DataFrame containing predictive power results.
        output_dir (str): Directory to save the scatter plots.

    Returns:
        None
    """
    # Prepare output path
    output_path = os.path.join(output_dir, f"{title} scatterplot.png")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    plt.figure(figsize=(12, 8))
    for stock, group in data.groupby('stock'):
        plt.plot(group[x_variable], group[y_variable], marker='o', label=stock)
    plt.xlabel(f"{x_variable}")
    plt.ylabel(f"{y_variable}")
    plt.title(f"{title}")
    plt.legend(title='Stock')
    plt.grid(True)
    plt.savefig(output_path)
    plt.close()
    print(f"Scatter plot saved to {output_path}.")



In [10]:
summary_df = pd.read_excel("../data/predictive_regression_results/predictive_power_summary.xlsx", sheet_name = 1)
visualize_heatmap(summary_df, title = "predictive_power_horizon_5")

/var/folders/yj/jjmvh7dd6sx95p4gzltx4j2r0000gn/T/ipykernel_11628/3773728321.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annotation_data = heatmap_data.applymap(lambda x: f"{x:.2e}")


Heatmap saved to ../results/predictive_power_horizon_5.png.


In [15]:
file_paths = sorted(glob.glob("../data/contemporaneous_regression_results/*_result.csv"))
contemporaneous_regression_results = []

for file_path in file_paths:
    date = file_path.split('/')[-1].split('_')[0]  # Extract date from filename
    daily_result = pd.read_csv(file_path)
    daily_result['date'] = date  # Add trading day column
    contemporaneous_regression_results.append(daily_result)

contemporaneous_impact_combined = pd.concat(contemporaneous_regression_results, ignore_index=True)
contemporaneous_impact_combined = contemporaneous_impact_combined.loc[:, ~contemporaneous_impact_combined.columns.str.contains('^Unnamed')]

visualize_scatter_plot(contemporaneous_impact_combined, x_variable="date", y_variable="r2")


Scatter plot saved to ../results/scatter_plot scatterplot.png.


In [18]:
file_paths = sorted(glob.glob("../data/predictive_regression_results/*_result.csv"))
contemporaneous_regression_results = []

for file_path in file_paths:
    date = file_path.split('/')[-1].split('_')[0]  # Extract date from filename
    daily_result = pd.read_csv(file_path)
    daily_result['date'] = date  # Add trading day column
    contemporaneous_regression_results.append(daily_result)


predictive_power_combined = pd.concat(contemporaneous_regression_results, ignore_index=True)
predictive_power_combined  = predictive_power_combined .loc[:, ~predictive_power_combined .columns.str.contains('^Unnamed')]

for horizon in [1, 5]:
    subset = predictive_power_combined[predictive_power_combined['horizon'] == horizon]
    visualize_scatter_plot(subset, title=f"predictive_power_horizon_{horizon}", x_variable="date", y_variable="r2")


Scatter plot saved to ../results/predictive_power_horizon_1 scatterplot.png.
Scatter plot saved to ../results/predictive_power_horizon_5 scatterplot.png.
